In [37]:
#default_exp encoder

In [38]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Encoder

> Architectures and functions for creating encoders that create the embeddings

In [39]:
#export
import pandas as pd
import numpy as np
from fastcore.all import *
from tsai.imports import *
from tsai.models.InceptionTimePlus import InceptionTimePlus
from tsai.data.preparation import SlidingWindow
from tsai.models.explainability import get_acts_and_grads
from tchub.dcae import DCAE_torch

### Dictionary to get the default backbone modules to get the embeddings from

In [40]:
#export
ENCODER_EMBS_MODULE_NAME = {
    InceptionTimePlus: 'backbone', # for mvp based models
    DCAE_torch: 'bottleneck.latent_in'
}

### Getting the embeddings (activations) from the encoder

In [41]:
#export
def get_enc_embs(X, enc_learn, module=None, cpu=False, average_seq_dim=True, to_numpy=True):
    """
        Get the embeddings of X from an encoder, passed in `enc_learn as a fastai 
        learner. By default, the embeddings are obtained from the last layer 
        before the model head, although any layer can be passed to `model`. 
        Input
        - `cpu`: Whether to do the model inference in cpu of gpu (GPU recommended)
        - `average_seq_dim`: Whether to aggregate the embeddings in the sequence dimensions
        - `to_numpy`: Whether to return the result as a numpy array (if false returns a tensor)
    """
    if cpu:
        enc_learn.dls.cpu()
        enc_learn.cpu()
    else:
        enc_learn.dls.cuda()
        enc_learn.cuda()
    aux_dl = enc_learn.dls.valid.new_dl(X=X)
    aux_dl.bs = enc_learn.dls.bs if enc_learn.dls.bs>0 else 64
    module = nested_attr(enc_learn.model, 
                         ENCODER_EMBS_MODULE_NAME[type(enc_learn.model)]) \
                if module is None else module
    embs = [get_acts_and_grads(model=enc_learn.model, 
                               modules=module, 
                               x=xb[0], cpu=cpu)[0] for xb in aux_dl]
    embs = to_concat(embs)
    if embs.ndim == 3 and average_seq_dim: embs = embs.mean(axis=2)
    if to_numpy: embs = embs.numpy() if cpu else embs.cpu().numpy()
    return embs

In [42]:
import wandb
from tchub.utils import *
wandb_api = wandb.Api()
enc_artifact = wandb_api.artifact('tchub/learner-mvp:run-tchub-3tipekxw')
enc_learner = enc_artifact.to_obj()
X = torch.rand(9, 1, 48)

In [43]:
#slow
%%time
embs = get_enc_embs(X, enc_learner, cpu=True)
test_eq(embs.shape[0], X.shape[0])
embs.shape, embs.__class__

/home/victor/env/lib/python3.8/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


CPU times: user 2min 39s, sys: 279 ms, total: 2min 39s
Wall time: 12.3 s


((9, 128), numpy.ndarray)

In [44]:
%%time
embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=False)
test_eq(embs.shape[0], X.shape[0])
embs.shape, embs.__class__, embs.device

CPU times: user 24.9 ms, sys: 7.87 ms, total: 32.8 ms
Wall time: 32.2 ms


(torch.Size([9, 128]), torch.Tensor, device(type='cuda', index=0))

In [45]:
%%time
embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=True)
test_eq(embs.shape[0], X.shape[0])
embs.shape, embs.__class__

CPU times: user 16.8 ms, sys: 3.94 ms, total: 20.7 ms
Wall time: 20 ms


((9, 128), numpy.ndarray)

## Export -

In [48]:
#hide
from nbdev.export import notebook2script
notebook2script()
beep(1)

Converted 01_dataset_artifact.ipynb.
Converted 02a_encoder_DCAE-torch.ipynb.
Converted 02b_encoder_MVP.ipynb.
Converted 04_dimensionality_reduction.ipynb.
Converted encoder.ipynb.
Converted index.ipynb.
Converted load.ipynb.
Converted utils.ipynb.
Converted visualization.ipynb.
